#### Carpeta del proyecto y almacenaje de ficheros

In [ ]:
# Importing required modules
import numpy as np
import pandas as pd
import utm
import requests
import os

# Project folder and file paths

# Define the name of the project folder and create it in case it does not exists yet
folder = "name_of_project_folder"

if not os.path.exists(folder):
    os.makedirs(folder)
    
print(f"The project will be saved in the folder: '{folder}'")

# Define the center coordinates of the project
latitud = 40.801706
longitud = -0.839873
print(f"The center coordinates of the project are the following: Lat '{latitud}', Lon '{longitud}'")

# Define the coordinates file name and path
file_name_coord = "coordinates_gmaps.csv"
file_path_coord = os.path.join(folder, file_name_coord)

# Define the roboflow results file name and path
file_name_results = "detections_rf.csv"
file_path_results = os.path.join(folder, file_name_results)

# Define the weather file name and path
file_name_weather = "weather_pvgis.csv"
file_path_weather = os.path.join(folder, file_name_weather)

# Define the pdc prediction file name and path
file_name_predictions_pdc = "predictions_pdc.csv"
file_path_predictions_pdc = os.path.join(folder, file_name_predictions_pdc)

# Define the eac prediction file name and path
file_name_predictions_eac = "predictions_pvlib_180_10.csv"
file_path_predictions_eac = os.path.join(folder, file_name_predictions_eac)


### Barrido imágenes GOOGLE MAPS STATIC

#### Definición de mallado y coordenadas

In [ ]:
# Python program to get google maps images of a specified location and a given area using 
# Google Static Maps API

# Define zoom an size of the images
zoom = 19  # zoom level
size = "400x400"  # image size
api_key_gmaps = "api_key_gmaps"  # enter your api key here

# Define center coordinates of the grid, equidistant from all edges of the grid
latitud = latitud
longitud = longitud
print(f"The center coordinates of the grid are the following: Lat '{latitud}', Lon '{longitud}'")

# Define area of the grid and related parameters
area_km2 = 1
area_m2 = area_km2 * 1000000
print(f"The defined area of the grid is '{area_km2}' km2")

square_size = 95 # depends on the zoom level of the image, defined in meters, measured in GMAPS (zoom 19 - size 95m)
grid_size = int (np.ceil((np.sqrt(area_m2) / square_size)))
num_images = np.square(grid_size) # Number of images for downloading and processing
area_evaluated_km2 = np.square(square_size * grid_size) / 1000000

print(f"The grid size is '{grid_size}' and the total number of images for processing will be '{num_images}'")
print(f"The total area evaluated is '{area_evaluated_km2}' km2")

In [ ]:
# Function to determine the grid coordinates depending on the desired sweep area

def crear_cuadricula(x_centro, y_centro, square_size, grid_size):
    # Calcular el desplazamiento para la cuadrícula
    desplazamiento = square_size * (grid_size // 2)

    # List to save the square centers
    centros = []

    # Loop to generate the grid square centers
    for i in range(grid_size):
        for j in range(grid_size):
            # Calculate the coordinates of the grid square centers
            x = x_centro + (i - (grid_size // 2)) * square_size
            y = y_centro + (j - (grid_size // 2)) * square_size
            
            # Save the coordinates of the grid square centers
            centros.append((x, y))
    
    return centros

# Obtaining UTM coordinates from the center of the defined main square 
utmcoord = utm.from_latlon(latitud, longitud)
x_centro = utmcoord[0]  # Primer valor del resultado (coordenada x)
y_centro = utmcoord[1]  # Segundo valor del resultado (coordenada y)


# Obtaining UTM coordinates of the grid square centers 
centros_cuadricula_utm = crear_cuadricula(x_centro, y_centro, square_size, grid_size)
# print(centros_cuadricula_utm)
print(f"The UTM coordinates of the grid images have been calculated")

# Function to convert UTM coordinates to latitud and longitude
def utm_a_latlon(x_utm, y_utm):
    # Define UTM zone and hemispheric letter
    zona_utm = 30
    letra_hemisferica = 'T'
    
    lat, lon = utm.to_latlon(x_utm, y_utm, zona_utm, letra_hemisferica)
    
    return lat, lon

# Convert UTM coordinates to latitud and longitude
centros_cuadricula_latlon = [utm_a_latlon(x_utm, y_utm) for x_utm, y_utm in centros_cuadricula_utm]
# print (centros_cuadricula_latlon)
print(f"The Lat, Lon coordinates of the grid images have been calculated")

# Save the coordinates grid square centers
coordinates = centros_cuadricula_latlon


In [ ]:
# Create the coordinates DataFrame
data_coord = {
    'center_lat': [latitud] * len(coordinates),
    'center_lon': [longitud] * len(coordinates),
    'image': [f'image_{i}' for i in range(len(coordinates))],
    'lat': [coord[0] for coord in coordinates],
    'lon': [coord[1] for coord in coordinates]
}
df_coord = pd.DataFrame(data_coord)

print(f"The coordinates dataframe has been created")

print(df_coord)

#### Almacenar fichero de coordenadas

In [ ]:
# Save the coordinates DataFrame

# Save the DataFrame in a CSV in its file path
df_coord.to_csv(file_path_coord, index=False)

print(f"CSV file saved in '{folder}/{file_name_coord}'")

#### Descarga de imágenes API GMAPS STATIC

**NO CORRER SI NO SE QUIERE HACER USO DE LA API DE GOOGLE MAPS STATIC**

In [ ]:
# Function to download images for each of the grid center
def download_images_for_grid(coordinates, zoom, size, api_key_gmaps):
       
    url = "https://maps.googleapis.com/maps/api/staticmap"
    for i, coord in enumerate(coordinates):
        center = f"{coord[0]}, {coord[1]}"
        r = requests.get(f'{url}?center={center}&zoom={zoom}&size={size}&maptype=satellite&key={api_key_gmaps}')
        image_path = os.path.join(folder, f'image_{i}.png')
        with open(image_path, 'wb') as f:
            f.write(r.content)

# Download images for each of the grid center
download_images_for_grid(coordinates, zoom, size, api_key)
print(f"Images saved in '{folder}'")

### Detección Roboflow

#### Definición del modelo

In [ ]:
# Importing required modules
import matplotlib.pyplot as plt
from skspatial.objects import Vector, Point
from skspatial.measurement import area_signed

from inference_sdk import InferenceHTTPClient, InferenceConfiguration

import supervision as sv
import cv2

# https://docs.roboflow.com/deploy/hosted-api/custom-models/object-detection
# https://universe.roboflow.com/whereareyousolarpanel/solar-pv-panel-detection/model/3

API_KEY_ROBOFLOW = "api_key_roboflow"
THRESHOLD = 0.7

# 0-Instalar Docker Desktop https://docs.docker.com/desktop/install/windows-install/
# 1-Abrir Docker (cerrar asegurandose que sigue en segundo plano (icono presente))
# 2-Lanzar máquina virtual desde cmd: inference server start
# 3-Lanzar script

client = InferenceHTTPClient(
    api_url="http://localhost:9001", # Opción docker desktop
    #api_url="https://detect.roboflow.com"   -> Acceso a RoboFlow hosted, opción de pago (1000 detecciones/mes)
    api_key=API_KEY_ROBOFLOW
)

client.configure(InferenceConfiguration(confidence_threshold=THRESHOLD))

def process_image(file_image):
    results = client.infer(file_image, model_id="solar-pv-panel-detection/3")
    labels = [item["class"] for item in results["predictions"]]
    detections = sv.Detections.from_inference(results)
    label_annotator = sv.LabelAnnotator()
    mask_annotator = sv.MaskAnnotator() # MaskAnnotator: draw mask - BoundingBoxAnnotator: draw bounding box

    image = cv2.imread(file_image) # to reed loacl file
    annotated_image = mask_annotator.annotate(scene=image, detections=detections)
    # annotated_image = label_annotator.annotate(scene=annotated_image, detections=detections, labels=labels)

    # Save annotated image
    output_image_path = f"{os.path.splitext(file_image)[0]}_w.png" # w for written (annotated) image
    cv2.imwrite(output_image_path, annotated_image)
    print(f"Processed {file_image}, saved annotated image to {output_image_path}")

    return results, detections

print(f"Ready for image processing")

#### Procesado de imágenes

In [ ]:
# Process multiple images locally saved

# DataFrame to save results
df_results_rf = pd.DataFrame(columns=["image_name", "coordinates", "area_rf", "orientation_rf"])

for i in range(num_images):
    file_image = os.path.join(folder, f"image_{i}.png")
    if os.path.exists(file_image):
        results, detections = process_image(file_image)

        for idx, prediction in enumerate(results['predictions']):
            points = pd.DataFrame(prediction['points'])

            pto_max_x = points.iloc[points.idxmax().x]
            pto_max_y = points.iloc[points.idxmax().y]
            pto_min_x = points.iloc[points.idxmin().x]
            pto_min_y = points.iloc[points.idxmin().y]

            dist_xmax_ymax = Point([pto_max_x.x, pto_max_x.y]).distance_point([pto_max_y.x, pto_max_y.y])
            dist_xmax_ymin = Point([pto_max_x.x, pto_max_x.y]).distance_point([pto_min_y.x, pto_min_y.y])

            v1 = Vector.from_points([pto_max_x.x, pto_max_x.y], [pto_max_y.x, pto_max_y.y])

            if dist_xmax_ymax < dist_xmax_ymin:
                v1 += 90

            orientation_rf = 180 + np.degrees(v1.angle_signed([0, 1])) # to use the orientation estimated geometrically
            
            area_rf = detections.area[idx]

            
            # Create a temporary DataFrame for new row
            df_temp = pd.DataFrame([{
                "image_name": file_image,
                "coordinates": coordinates[i],
                "area_rf": area_rf,
                "orientation_rf": orientation_rf,          
            }])

            # Concat temporary DataFrame with main df
            df_results_rf = pd.concat([df_results_rf, df_temp], ignore_index=True)

            #print(f"Image: {file_image}, Coordinates: {coordinates[i]}º, Area: {area_rf} m2")
    else:
        print(f"File {file_image} does not exist")

print(f"The Roboflow results dataframe has been created")

# Show the results DataFrame
print(df_results_rf)

# print (results)
# print (detections)

#### Almacenar fichero de resultados

In [ ]:
# Save the Roboflow results DataFrame

# Save the DataFrame in a CSV in its file path
df_results_rf.to_csv(file_path_results, index=False)

print(f"CSV file saved in '{folder}/{file_name_results}'")

### Estimación generación PVLIB

#### Definición de parámetros

In [ ]:
import pvlib
from pvlib.pvsystem import PVSystem, Array, FixedMount
from pvlib.location import Location
from pvlib.modelchain import ModelChain

timezone = 'Europe/Madrid'
temperature_model_parameters = pvlib.temperature.TEMPERATURE_MODEL_PARAMETERS['sapm']['insulated_back_glass_polymer'] #change temperature if desired
# insulated_back_glass_polymer
# open_rack_glass_polymer
# open_rack_glass_glass
# close_mount_glass_glass

module_eff = 0.2
gamma = -0.004 # module temperature coefficient for Power loss reduction
eta_inv = 0.98 # inverter efficiency
system_loss = 0.15  # general system losses (dust, electrical, missmatch, shadowing ...)

area_ratio = 23.3 # Ratio between the output area of roboflow and the measured area in GMAPS, pixels/m

orientation_deg = 180
inclination_deg = 10

print(f"PV parameters set")

#### Estimación de la potencia instalada

In [ ]:
# Duplicate results DataFrame from Roboflow
#df_predictions_pdc = df_results_rf #copy df
df_predictions_pdc = pd.read_csv(file_path_results) #import df locally saved


# Add columns for area in m2 and installed pdc
df_predictions_pdc["area_m2"] = 0.0
df_predictions_pdc["pdc_kW"] = 0.0

# Define for each row the area and pdc
for idx, row in df_predictions_pdc.iterrows():

    df_predictions_pdc.at[idx, "area_m2"] = df_predictions_pdc.at[idx, "area_rf"] / area_ratio
    df_predictions_pdc.at[idx, "pdc_kW"] = df_predictions_pdc.at[idx, "area_m2"] * module_eff

print(f"The prediction results dataframe with Pdc has been created")

# Show the DataFrame results for installed pdc
print(df_predictions_pdc)


#### Almacenar fichero pdc

In [ ]:
# Save the Pdc predictions DataFrame

# Save the DataFrame in a CSV in its file path
df_predictions_pdc.to_csv(file_path_predictions_pdc, index=False)

print(f"CSV file saved in '{folder}/{file_name_predictions_pdc}'")

#### Descarga de datos meteorológicos de PVGIS

**NO CORRER SI NO SE QUIERE HACER USO DE LA API DE PVGIS**

In [ ]:
weather_pvgis = pvlib.iotools.get_pvgis_tmy(latitud, longitud)[0]

#print (weather_pvgis)
print(f"Weather data from PVGIS has been downloaded")

#### Almacenar fichero meteo

In [ ]:
# Save the PVGis weather data

# Save the DataFrame in a CSV in its file path
weather_pvgis.to_csv(file_path_weather)

print(f"CSV file saved in '{folder}/{file_name_weather}'")

#### Estimación energética

In [ ]:
# Duplicate DataFrame with results from pdc
#df_predictions_eac = df_predictions_pdc #copy df
df_predictions_eac = pd.read_csv(file_path_predictions_pdc) #import df locally saved

# weather_imp = pd.read_csv(file_path_weather) # import pvgis file 

# Add columns for orientation, inclination and annual energy
df_predictions_eac["orientation_deg"] = orientation_deg #change for orientation_rf if desired
df_predictions_eac["inclination_deg"] = inclination_deg
df_predictions_eac["annual_energy_kWh"] = 0.0

for idx, row in df_predictions_eac.iterrows():

    #latitude, longitude = row["coordinates"]   
    surface_tilt = row["inclination_deg"]
    surface_azimuth = row["orientation_deg"] 
    pdc = row["pdc_kW"]
  
    module = {'pdc0': pdc, 'gamma_pdc': gamma}
    inverter = {'pdc0': pdc, 'eta_inv_nom': eta_inv}

    location = Location(
        latitude=latitud, #coordinates from the center of GMAPS sweep
        longitude=longitud,
        tz=timezone,
    )

    #weather = pvlib.iotools.get_pvgis_tmy(latitude, longitude)
    #If this option is used, many pvgis queries are made and for a small area it is better to take only the center of the main square coordinates
    
    weather = weather_pvgis # need to run the apicorrer la api
    #weather = weather_imp #import df saved, not working by now
    
    mount = FixedMount(surface_tilt=surface_tilt, surface_azimuth=surface_azimuth)
    
    array = Array(
        mount=mount,
        module_parameters=module,
        temperature_model_parameters=temperature_model_parameters,
    )

    system = PVSystem(arrays=[array], inverter_parameters=inverter)

    mc = ModelChain(system, location, aoi_model='physical', spectral_model='no_loss')
    mc.run_model(weather)

    annual_energy_kWh = mc.results.ac.sum() * (1 - system_loss)

    df_predictions_eac.at[idx, "annual_energy_kWh"] = annual_energy_kWh

print(f"The prediction results dataframe with annual Eac has been created")

# Show the DataFrame with annual energy results
print(df_predictions_eac)


#### Almacenar fichero Eac

In [ ]:
# Save the PVlib predictions DataFrame

# Save the DataFrame in a CSV in its file path
df_predictions_eac.to_csv(file_path_predictions_eac, index=False)

print(f"CSV file saved in '{folder}/{file_name_predictions_eac}'")

# NOTAS